In [2]:
!pip install SpeechRecognition gTTS pydub



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 53.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 6.4 MB/s eta 0:00:00
  Attempting uninstall: click
    Found existing installation: click 8.2.0
    Uninstalling click-8.2.0:
      Successfully uninstalled click-8.2.0


In [3]:
import speech_recognition as sr
from gtts import gTTS
from pydub import AudioSegment
import re
import io
from IPython.display import Audio, display

In [4]:
def parse_expression(text):
    text = text.lower()
    word_to_num = {
        'zero': '0', 'one': '1', 'two': '2', 'three': '3', 'four': '4',
        'five': '5', 'six': '6', 'seven': '7', 'eight': '8', 'nine': '9',
        'plus': '+', 'minus': '-', 'times': '*', 'divided by': '/', 'divide': '/'
    }
    for word, num in word_to_num.items():
        text = text.replace(word, num)
    pattern = r'(\d+\.?\d*)\s*([+\-*/])\s*(\d+\.?\d*)'
    match = re.match(pattern, text)
    if not match:
        return None, "Invalid expression. Please say something like 'two plus three'."
    num1, operator, num2 = match.groups()
    try:
        num1 = float(num1)
        num2 = float(num2)
        return (num1, operator, num2), None
    except ValueError:
        return None, "Error parsing numbers."

def calculate(num1, operator, num2):
    try:
        if operator == '+':
            return num1 + num2, None
        elif operator == '-':
            return num1 - num2, None
        elif operator == '*':
            return num1 * num2, None
        elif operator == '/':
            if num2 == 0:
                return None, "Division by zero is not allowed."
            return num1 / num2, None
        else:
            return None, "Invalid operator."
    except Exception as e:
        return None, f"Calculation error: {str(e)}"

In [5]:
def convert_to_wav(input_file):
    audio = AudioSegment.from_file(input_file)
    output_file = "converted_audio.wav"
    audio = audio.set_frame_rate(16000).set_channels(1).set_sample_width(2)
    audio.export(output_file, format="wav")
    return output_file

In [6]:

def process_audio(audio_file):
    recognizer = sr.Recognizer()
    with sr.AudioFile(audio_file) as source:
        audio = recognizer.record(source)
        try:
            text = recognizer.recognize_google(audio)
            return text, None
        except sr.UnknownValueError:
            return None, "Could not understand the audio."
        except sr.RequestError as e:
            return None, f"Speech recognition error: {e}"

In [7]:
def text_to_speech(text):
    tts = gTTS(text=text, lang='en')
    audio_buffer = io.BytesIO()
    tts.write_to_fp(audio_buffer)
    audio_buffer.seek(0)
    return audio_buffer

In [8]:
from google.colab import files
uploaded = files.upload()
input_file = list(uploaded.keys())[0]
audio_file = convert_to_wav(input_file)


Saving saloni.mp3 to saloni.mp3


In [9]:
text, error = process_audio(audio_file)
if error:
    result_text = error
else:
    expression, error = parse_expression(text)
    if error:
        result_text = error
    else:
        num1, operator, num2 = expression
        result, calc_error = calculate(num1, operator, num2)
        if calc_error:
            result_text = calc_error
        else:
            result_text = f"The result is {result}"

In [10]:
audio_buffer = text_to_speech(result_text)
audio = Audio(audio_buffer.read(), autoplay=True)
display(audio)
